In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from imutils import paths
import cv2 as cv

from tensorflow.keras.applications import MobileNetV2 
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

from imutils.video import VideoStream
import imutils

In [2]:
dataset = '/Users/kunal/Desktop/Clocky/dataset'
imagePaths = list(paths.list_images(dataset))

In [3]:
imagePaths

['/Users/kunal/Desktop/Clocky/dataset/with_brush/77.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/162.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/176.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/88.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/348.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/406.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/412.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/374.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/228.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/214.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/215.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/229.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/413.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/375.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/361.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/407.jpg',
 '/Users/kunal/Desktop/Clocky/dataset/with_brush/349.jpg',

In [4]:
data = []
labels = []

for i in imagePaths:
    label = i.split(os.path.sep)[-2]
    labels.append(label)
    image = load_img(i, target_size = (224, 224))
    image = img_to_array(image)
    image = preprocess_input(image)
    data.append(image)

In [5]:
data = np.array(data, dtype = 'float32')
labels = np.array(labels)

In [6]:
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels = to_categorical(labels)

In [7]:
train_X, test_X, train_Y, test_Y = train_test_split(data, labels, test_size = 0.20, random_state = 10, stratify = labels)

In [8]:
aug = ImageDataGenerator(rotation_range=20, zoom_range= 0.15, width_shift_range= 0.2, shear_range= 0.15, horizontal_flip= True, vertical_flip= True, fill_mode='nearest')

In [9]:
baseModel = MobileNetV2(weights='imagenet', include_top=False, input_tensor= Input(shape = (224,224,3)))

In [10]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size = (7,7))(headModel)
headModel = Flatten(name = 'Flatten')(headModel)
headModel = Dense(128, activation = 'relu')(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation = 'softmax')(headModel)

model = Model(inputs = baseModel.input, outputs = headModel)

In [11]:
for layer in baseModel.layers:
    layer.trainable = False

In [12]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [13]:
learning_rate = 0.001
Epochs = 20
batch_size = 12

opt = Adam(lr=learning_rate, decay = learning_rate/Epochs)

model.compile(loss = 'binary_crossentropy', optimizer = opt, metrics = ['accuracy'])

H = model.fit(
    aug.flow(train_X, train_Y, batch_size = batch_size),
    steps_per_epoch=len(train_X)//batch_size,
    validation_data=(test_X, test_Y),
    validation_steps=len(test_X)//batch_size,
    epochs = Epochs
)

Epoch 1/20
34/34 [==============================] - 11s 268ms/step - loss: 0.7104 - accuracy: 0.6320 - val_loss: 0.1672 - val_accuracy: 0.9515
Epoch 2/20
34/34 [==============================] - 8s 237ms/step - loss: 0.3139 - accuracy: 0.8726 - val_loss: 0.1238 - val_accuracy: 0.9612
Epoch 3/20
34/34 [==============================] - 9s 249ms/step - loss: 0.2039 - accuracy: 0.9362 - val_loss: 0.1096 - val_accuracy: 0.9515
Epoch 4/20
34/34 [==============================] - 9s 264ms/step - loss: 0.2239 - accuracy: 0.9167 - val_loss: 0.0885 - val_accuracy: 0.9612
Epoch 5/20
34/34 [==============================] - 9s 258ms/step - loss: 0.2475 - accuracy: 0.9129 - val_loss: 0.1033 - val_accuracy: 0.9612
Epoch 6/20
34/34 [==============================] - 9s 264ms/step - loss: 0.1834 - accuracy: 0.9270 - val_loss: 0.0834 - val_accuracy: 0.9612
Epoch 7/20
34/34 [==============================] - 9s 262ms/step - loss: 0.2299 - accuracy: 0.9270 - val_loss: 0.1018 - val_accuracy: 0.9515
Epoch

In [14]:
model.save('clocky.model')

INFO:tensorflow:Assets written to: clocky.model/assets


In [15]:
predict = model.predict(test_X, batch_size=batch_size)
predict = np.argmax(predict, axis = 1)
print(classification_report(test_Y.argmax(axis=1), predict, target_names = lb.classes_))

               precision    recall  f1-score   support

   with_brush       0.98      0.94      0.96        51
without_brush       0.94      0.98      0.96        52

     accuracy                           0.96       103
    macro avg       0.96      0.96      0.96       103
 weighted avg       0.96      0.96      0.96       103

